In [56]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier 


train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

test_id=pd.DataFrame(test_df["ID_code"])

# var3 피처 값 대체 및 ID 피처 드롭
train_df.drop('ID_code',axis=1 , inplace=True)
test_df.drop('ID_code',axis=1 , inplace=True)


train = train_df.iloc[:, 1:]
y_labels = train_df.iloc[:, -0]

In [22]:
print(train.shape)
print(test_df.shape)
print(y_labels.shape)

(200000, 200)
(200000, 200)
(200000,)


In [28]:
idx = features = train_df.columns.values[1:201]
for df in [test_df, train]:
    df['sum'] = df[idx].sum(axis=1)  
    df['min'] = df[idx].min(axis=1)
    df['max'] = df[idx].max(axis=1)
    df['mean'] = df[idx].mean(axis=1)
    df['std'] = df[idx].std(axis=1)
    df['skew'] = df[idx].skew(axis=1)
    df['kurt'] = df[idx].kurtosis(axis=1)
    df['med'] = df[idx].median(axis=1)

In [29]:
print(train.shape)
print(test_df.shape)
print(y_labels.shape)

(200000, 208)
(200000, 208)
(200000,)


In [32]:
new_train_df = pd.read_csv("X_features.csv")
new_test_df = pd.read_csv("new_test.csv")
print("new_train.shape: {}".format(new_train_df.shape))
print("new_test.shape: {}".format(new_test_df.shape))

new_train.shape: (200000, 212)
new_test.shape: (200000, 212)


In [37]:
new_train_df=new_train_df.iloc[:,200:]
new_test_df=new_test_df.iloc[:,200:]

In [38]:
new_train_df = pd.concat([train,new_train_df],axis=1)
new_test_df = pd.concat([test_df,new_test_df],axis=1)
new_train_df.to_csv("new_train.csv", index=False)
new_test_df.to_csv("new_test.csv", index=False)

In [69]:
from imblearn.over_sampling import RandomOverSampler

smote = RandomOverSampler(random_state=0)
X_train_over, y_train_over = smote.fit_sample(X_train, y_train)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (160000, 220) (160000,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (287832, 220) (287832,)
SMOTE 적용 후 레이블 값 분포: 
 1    143916
0    143916
dtype: int64


In [70]:
X_train, X_test, y_train, y_test = train_test_split(new_train_df, y_labels,
                                                    test_size=0.2, random_state=0)

from imblearn.over_sampling import RandomOverSampler

smote = RandomOverSampler(random_state=0)
X_train_over, y_train_over = smote.fit_sample(X_train, y_train)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

lgbm_clf = LGBMClassifier(    
    n_estimators=10000,
    bagging_fraction= 0.3,
    boost_from_average= False,
    boost= 'gbdt',
    feature_fraction= 0.041,
    learning_rate= 0.03,
    max_depth= -1,
    metric='logloss',
    min_data_in_leaf= 80,
    min_sum_hessian_in_leaf= 10.0,
    num_leaves= 13,
    num_threads= 8,
    tree_learner= 'serial',
    objective= 'binary', 
    verbosity= -1
)

evals = [(X_test, y_test)]
lgbm_clf.fit(X_train_over, y_train_over, early_stopping_rounds=100, eval_metric="logloss", eval_set=evals,
                verbose=True)

lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:,1],average='macro')
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (160000, 220) (160000,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (287832, 220) (287832,)
SMOTE 적용 후 레이블 값 분포: 
 1    143916
0    143916
dtype: int64
[1]	valid_0's binary_logloss: 0.692563
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.691609
[3]	valid_0's binary_logloss: 0.690713
[4]	valid_0's binary_logloss: 0.689974
[5]	valid_0's binary_logloss: 0.689201
[6]	valid_0's binary_logloss: 0.688406
[7]	valid_0's binary_logloss: 0.687669
[8]	valid_0's binary_logloss: 0.686963
[9]	valid_0's binary_logloss: 0.685997
[10]	valid_0's binary_logloss: 0.685482
[11]	valid_0's binary_logloss: 0.684309
[12]	valid_0's binary_logloss: 0.683702
[13]	valid_0's binary_logloss: 0.682937
[14]	valid_0's binary_logloss: 0.682268
[15]	valid_0's binary_logloss: 0.681375
[16]	valid_0's binary_logloss: 0.680714
[17]	valid_0's binary_logloss: 0.679844
[18]	valid_0's binary_logloss: 0.678759
[19]	valid_0's binary_logloss: 0.678085
[20]	valid_0's b

[202]	valid_0's binary_logloss: 0.570628
[203]	valid_0's binary_logloss: 0.570116
[204]	valid_0's binary_logloss: 0.569713
[205]	valid_0's binary_logloss: 0.569405
[206]	valid_0's binary_logloss: 0.569011
[207]	valid_0's binary_logloss: 0.56848
[208]	valid_0's binary_logloss: 0.568214
[209]	valid_0's binary_logloss: 0.567877
[210]	valid_0's binary_logloss: 0.567399
[211]	valid_0's binary_logloss: 0.566994
[212]	valid_0's binary_logloss: 0.566659
[213]	valid_0's binary_logloss: 0.566233
[214]	valid_0's binary_logloss: 0.565713
[215]	valid_0's binary_logloss: 0.56547
[216]	valid_0's binary_logloss: 0.565247
[217]	valid_0's binary_logloss: 0.564802
[218]	valid_0's binary_logloss: 0.564339
[219]	valid_0's binary_logloss: 0.563803
[220]	valid_0's binary_logloss: 0.563259
[221]	valid_0's binary_logloss: 0.562837
[222]	valid_0's binary_logloss: 0.562595
[223]	valid_0's binary_logloss: 0.562207
[224]	valid_0's binary_logloss: 0.561937
[225]	valid_0's binary_logloss: 0.56141
[226]	valid_0's bin

[415]	valid_0's binary_logloss: 0.505508
[416]	valid_0's binary_logloss: 0.505115
[417]	valid_0's binary_logloss: 0.50494
[418]	valid_0's binary_logloss: 0.504614
[419]	valid_0's binary_logloss: 0.504357
[420]	valid_0's binary_logloss: 0.504158
[421]	valid_0's binary_logloss: 0.503988
[422]	valid_0's binary_logloss: 0.503706
[423]	valid_0's binary_logloss: 0.503561
[424]	valid_0's binary_logloss: 0.503344
[425]	valid_0's binary_logloss: 0.503007
[426]	valid_0's binary_logloss: 0.502804
[427]	valid_0's binary_logloss: 0.502628
[428]	valid_0's binary_logloss: 0.502444
[429]	valid_0's binary_logloss: 0.502196
[430]	valid_0's binary_logloss: 0.502006
[431]	valid_0's binary_logloss: 0.501815
[432]	valid_0's binary_logloss: 0.501605
[433]	valid_0's binary_logloss: 0.501409
[434]	valid_0's binary_logloss: 0.501228
[435]	valid_0's binary_logloss: 0.50096
[436]	valid_0's binary_logloss: 0.500739
[437]	valid_0's binary_logloss: 0.500427
[438]	valid_0's binary_logloss: 0.500151
[439]	valid_0's bi

[630]	valid_0's binary_logloss: 0.465868
[631]	valid_0's binary_logloss: 0.465667
[632]	valid_0's binary_logloss: 0.465512
[633]	valid_0's binary_logloss: 0.465398
[634]	valid_0's binary_logloss: 0.465304
[635]	valid_0's binary_logloss: 0.465213
[636]	valid_0's binary_logloss: 0.465036
[637]	valid_0's binary_logloss: 0.46495
[638]	valid_0's binary_logloss: 0.464823
[639]	valid_0's binary_logloss: 0.464704
[640]	valid_0's binary_logloss: 0.464526
[641]	valid_0's binary_logloss: 0.464393
[642]	valid_0's binary_logloss: 0.464175
[643]	valid_0's binary_logloss: 0.464015
[644]	valid_0's binary_logloss: 0.463864
[645]	valid_0's binary_logloss: 0.463717
[646]	valid_0's binary_logloss: 0.463556
[647]	valid_0's binary_logloss: 0.463412
[648]	valid_0's binary_logloss: 0.463281
[649]	valid_0's binary_logloss: 0.463181
[650]	valid_0's binary_logloss: 0.463042
[651]	valid_0's binary_logloss: 0.462916
[652]	valid_0's binary_logloss: 0.462795
[653]	valid_0's binary_logloss: 0.46266
[654]	valid_0's bi

[843]	valid_0's binary_logloss: 0.439191
[844]	valid_0's binary_logloss: 0.439084
[845]	valid_0's binary_logloss: 0.438994
[846]	valid_0's binary_logloss: 0.438882
[847]	valid_0's binary_logloss: 0.438793
[848]	valid_0's binary_logloss: 0.43866
[849]	valid_0's binary_logloss: 0.438565
[850]	valid_0's binary_logloss: 0.438482
[851]	valid_0's binary_logloss: 0.438387
[852]	valid_0's binary_logloss: 0.438276
[853]	valid_0's binary_logloss: 0.438169
[854]	valid_0's binary_logloss: 0.43799
[855]	valid_0's binary_logloss: 0.437861
[856]	valid_0's binary_logloss: 0.437738
[857]	valid_0's binary_logloss: 0.437668
[858]	valid_0's binary_logloss: 0.437563
[859]	valid_0's binary_logloss: 0.437471
[860]	valid_0's binary_logloss: 0.437383
[861]	valid_0's binary_logloss: 0.437272
[862]	valid_0's binary_logloss: 0.437181
[863]	valid_0's binary_logloss: 0.437103
[864]	valid_0's binary_logloss: 0.437022
[865]	valid_0's binary_logloss: 0.436907
[866]	valid_0's binary_logloss: 0.43682
[867]	valid_0's bin

...

[9062]	valid_0's binary_logloss: 0.277547
[9063]	valid_0's binary_logloss: 0.27754
[9064]	valid_0's binary_logloss: 0.27753
[9065]	valid_0's binary_logloss: 0.277528
[9066]	valid_0's binary_logloss: 0.277521
[9067]	valid_0's binary_logloss: 0.277516
[9068]	valid_0's binary_logloss: 0.277504
[9069]	valid_0's binary_logloss: 0.277497
[9070]	valid_0's binary_logloss: 0.277492
[9071]	valid_0's binary_logloss: 0.277479
[9072]	valid_0's binary_logloss: 0.277476
[9073]	valid_0's binary_logloss: 0.277466
[9074]	valid_0's binary_logloss: 0.27745
[9075]	valid_0's binary_logloss: 0.277439
[9076]	valid_0's binary_logloss: 0.277429
[9077]	valid_0's binary_logloss: 0.277418
[9078]	valid_0's binary_logloss: 0.277412
[9079]	valid_0's binary_logloss: 0.277402
[9080]	valid_0's binary_logloss: 0.277389
[9081]	valid_0's binary_logloss: 0.277377
[9082]	valid_0's binary_logloss: 0.277375
[9083]	valid_0's binary_logloss: 0.277365
[9084]	valid_0's binary_logloss: 0.277357
[9085]	valid_0's binary_logloss: 0.27

[9259]	valid_0's binary_logloss: 0.275969
[9260]	valid_0's binary_logloss: 0.27596
[9261]	valid_0's binary_logloss: 0.275951
[9262]	valid_0's binary_logloss: 0.275948
[9263]	valid_0's binary_logloss: 0.275946
[9264]	valid_0's binary_logloss: 0.275935
[9265]	valid_0's binary_logloss: 0.275925
[9266]	valid_0's binary_logloss: 0.27592
[9267]	valid_0's binary_logloss: 0.275913
[9268]	valid_0's binary_logloss: 0.275898
[9269]	valid_0's binary_logloss: 0.275885
[9270]	valid_0's binary_logloss: 0.275875
[9271]	valid_0's binary_logloss: 0.275869
[9272]	valid_0's binary_logloss: 0.275863
[9273]	valid_0's binary_logloss: 0.275858
[9274]	valid_0's binary_logloss: 0.275845
[9275]	valid_0's binary_logloss: 0.275841
[9276]	valid_0's binary_logloss: 0.275832
[9277]	valid_0's binary_logloss: 0.275828
[9278]	valid_0's binary_logloss: 0.275822
[9279]	valid_0's binary_logloss: 0.275817
[9280]	valid_0's binary_logloss: 0.275807
[9281]	valid_0's binary_logloss: 0.275802
[9282]	valid_0's binary_logloss: 0.2

[9460]	valid_0's binary_logloss: 0.27446
[9461]	valid_0's binary_logloss: 0.274453
[9462]	valid_0's binary_logloss: 0.274452
[9463]	valid_0's binary_logloss: 0.274445
[9464]	valid_0's binary_logloss: 0.274436
[9465]	valid_0's binary_logloss: 0.274431
[9466]	valid_0's binary_logloss: 0.274425
[9467]	valid_0's binary_logloss: 0.274424
[9468]	valid_0's binary_logloss: 0.274414
[9469]	valid_0's binary_logloss: 0.274408
[9470]	valid_0's binary_logloss: 0.274405
[9471]	valid_0's binary_logloss: 0.2744
[9472]	valid_0's binary_logloss: 0.274391
[9473]	valid_0's binary_logloss: 0.274389
[9474]	valid_0's binary_logloss: 0.274385
[9475]	valid_0's binary_logloss: 0.27438
[9476]	valid_0's binary_logloss: 0.27437
[9477]	valid_0's binary_logloss: 0.274361
[9478]	valid_0's binary_logloss: 0.274351
[9479]	valid_0's binary_logloss: 0.274347
[9480]	valid_0's binary_logloss: 0.27434
[9481]	valid_0's binary_logloss: 0.274334
[9482]	valid_0's binary_logloss: 0.274331
[9483]	valid_0's binary_logloss: 0.27432

[9673]	valid_0's binary_logloss: 0.272978
[9674]	valid_0's binary_logloss: 0.27297
[9675]	valid_0's binary_logloss: 0.272965
[9676]	valid_0's binary_logloss: 0.272955
[9677]	valid_0's binary_logloss: 0.272941
[9678]	valid_0's binary_logloss: 0.27294
[9679]	valid_0's binary_logloss: 0.272932
[9680]	valid_0's binary_logloss: 0.272924
[9681]	valid_0's binary_logloss: 0.272916
[9682]	valid_0's binary_logloss: 0.272909
[9683]	valid_0's binary_logloss: 0.272906
[9684]	valid_0's binary_logloss: 0.272905
[9685]	valid_0's binary_logloss: 0.272902
[9686]	valid_0's binary_logloss: 0.272898
[9687]	valid_0's binary_logloss: 0.272889
[9688]	valid_0's binary_logloss: 0.272885
[9689]	valid_0's binary_logloss: 0.272874
[9690]	valid_0's binary_logloss: 0.272866
[9691]	valid_0's binary_logloss: 0.272864
[9692]	valid_0's binary_logloss: 0.27286
[9693]	valid_0's binary_logloss: 0.27285
[9694]	valid_0's binary_logloss: 0.272846
[9695]	valid_0's binary_logloss: 0.272841
[9696]	valid_0's binary_logloss: 0.272

[9890]	valid_0's binary_logloss: 0.271378
[9891]	valid_0's binary_logloss: 0.271373
[9892]	valid_0's binary_logloss: 0.271358
[9893]	valid_0's binary_logloss: 0.271346
[9894]	valid_0's binary_logloss: 0.271343
[9895]	valid_0's binary_logloss: 0.271339
[9896]	valid_0's binary_logloss: 0.271329
[9897]	valid_0's binary_logloss: 0.271326
[9898]	valid_0's binary_logloss: 0.271317
[9899]	valid_0's binary_logloss: 0.271311
[9900]	valid_0's binary_logloss: 0.271302
[9901]	valid_0's binary_logloss: 0.271298
[9902]	valid_0's binary_logloss: 0.271294
[9903]	valid_0's binary_logloss: 0.271285
[9904]	valid_0's binary_logloss: 0.271273
[9905]	valid_0's binary_logloss: 0.271262
[9906]	valid_0's binary_logloss: 0.271257
[9907]	valid_0's binary_logloss: 0.27125
[9908]	valid_0's binary_logloss: 0.271238
[9909]	valid_0's binary_logloss: 0.271232
[9910]	valid_0's binary_logloss: 0.271225
[9911]	valid_0's binary_logloss: 0.271219
[9912]	valid_0's binary_logloss: 0.271217
[9913]	valid_0's binary_logloss: 0.

In [71]:
y_test_pred=lgbm_clf.predict_proba(new_test_df)[:,1]

In [73]:
sub_df1 = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df1["target"] = y_test_pred
sub_df1.to_csv("20190328_3.csv", index=False)